In [16]:
from pyspark.sql import SparkSession
import os
import matplotlib.pyplot as plt


In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0,io.delta:delta-core_2.12:2.0.1 pyspark-shell'
spark = SparkSession \
            .builder \
            .appName("water_test") \
            .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") \
            .master("local[*]") \
            .getOrCreate()

# Check LT & SE refine data

In [14]:
se_data = spark.read.format('parquet').load('/home/jovyan/dags/output/raw/bronze_se_data')


-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 monitoringSiteIdentifier           | SE616397-135001                                                                                                                                          
 parameterWaterBodyCategory         | RW                                                                                                                                                       
 observedPropertyDeterminandCode    | EEA_3164-01-0                                                                                                                                            
 observedPropertyDeterminandLabel   | Chlorophyll a                                                                                                                                            
 procedureAnalysedMatrix            | W 